# ANOVA docomposition of a general function

Functional ANOVA decomposition represents a high-dimensional function as a function of the form
$$f(x_1, x_2, \ldots, x_D) = f_0 + \sum_{i=1}^D f_i(x_i) + \sum_{i<j} f_{ij}(x_i, x_j) + \sum_{i<j<k} f_{ijk}(x_i, x_j, x_k) + \cdots + f_{1,\ldots,D}(x_1, \ldots, x_D).$$


Let's first talk about how to compute the ANOVA components in general and then we focus on how to do it for $f(x_1, \ldots, x_D)$ which is evaluated as a mean of a Gaussian process.

Note: without the loss of generality, we assumy that the input is restricted to a hypercube $[0,1]^D$. It will significanlty simplify the notation. We can normalize the input whe computing.

The first step is to choose the projection operator $P$:
$$Pf := \int_{[0,1]}f(x)d\mu(x)$$
We are going to use the projection operator $P$ using Lebegue measure $Pf := \int_{[0,1]} f(x) dx$, so all intergrals should work out as expected.

## Two-dimensional case

Now we use the projection operator to define the constant and the main effects. We assume $D=2$ for now and then introduce some more notation to generalize:
\begin{align}
f_0 & = \int_{[0, 1]} \int_{[0,1]} f(x_1, x_2) dx_1 dx_2 \\
f_1 (x_1) & = \int_{[0, 1]} \Bigl(f(x_1, x_2) - f_0 \Bigr) dx_2  \\
f_2 (x_2) & = \int_{[0, 1]} \Bigl(f(x_1, x_2) - f_0 \Bigr) dx_1  
\end{align}

The interaction effect $f_{1,2}(x_1,x_2)$ is defined as the remainder to make the ANOVA decomposition to work out correctly:
$$f_{1,2}(x_1,x_2) = f(x_1, x_2) - f_0 - f_1(x_1) - f_2(x_2).$$

The **total variance** (TV) of the predictor is defined as
$$\sigma^2(f) := \int_{[0, 1]} \int_{[0, 1]} (f(x_1, x_2) - f_0)^2 d x_1 d x_2$$

One can show that TV is decomposible into the sum of variances of main effects and interactions defined above:
\begin{align}
\sigma_1^2(f_1) &:= \int_{[0, 1]} (f_1(x_1))^2 d x_1 \\
\sigma_2^2(f_2) &:= \int_{[0, 1]} (f_2(x_2))^2 d x_2 \\
\sigma_{1,2}^2(f_{1,2}) &:= \int_{[0, 1]} \int_{[0, 1]} \left(f_{1,2}(x_1, x_2) - f_0 - \left[f_1(x_1)-f_0\right] - \left[f_2(x_2)-f_0\right]\right)^2 dx_1 dx_2 \\ 
\sigma^2(f) &= \sigma_1^2(f_1) + \sigma_2^2(f_2) + \sigma_{1,2}^2 (f_{1,2}).
\end{align}

And so, by dividing individual variances by TV we can express these compoents as percentages.

## General case

Using subsets $u\subseteq \{1,\ldots, D\}$, we can establish a shorthand notation for ANOVA components, where $f_u$ and $x_u$ represents a subset of vector $x$ with components $x_i, i \in u$. The we have
\begin{align}
f(x_1, \ldots, x_D) &= \sum_{u \subseteq\{1\ldots,D\}} f_{u}(x_{u}),\\
f_{u}(x_{u}) &= \int_{[0,1]^{D-|u|}} \Bigl(f(x) - \sum_{v\subsetneq u}f_v(x_v)\Bigr)dx_{-u}\\
\sigma^2(f_u) &= \int_{[0,1]^D} \Bigl(f_u(x_u)\Bigr)^2dx \\
\sigma^2(f) &= \int_{[0,1]^D} \Bigl(f(x) - f_0\Bigr)^2 dx = \sum_{u \subseteq \{1,\ldots,D\}, u\neq\emptyset}\sigma^2(f_u).
\end{align}

## ANOVA for Gaussian process

For Bayesian optimization we are using the Gaussian proces defined by a parametrized RBF kernel of the form
$$
K(x,x'|\theta_0, \vec{\theta}_1) = \theta_0^2 \exp\Bigl(-(x-x')^t D(\vec{\theta}_1) (x-x'))\Bigr),
$$
where $[D(\vec{\theta}_1)]_{ij} = (\theta_{1,i}^2 + \epsilon)\delta_{ij}$ is a diagonal matrix of scaling coefficients, $\epsilon$ beeing a constant and $\delta_{ij}$ beeing Dirac-delta.
$$
K(x,x'|\theta_0, \vec{\theta}_1) = \theta_0^2\exp\Bigl(-\sum_{d=1}^D (\theta_{1d}^2+\epsilon)(x_d-x'_d)^2\Bigr) = \theta_0^2 \prod_{d=1}^D \exp \Bigl(-(\theta_{1d}^2+\epsilon)(x_d-x'_d)^2\Bigr) . 
$$

I drop $\theta$'s from the definition of $K$ for shorthand.

Let $\Sigma$ be the correlation matrix of the training set $\{(x_i, y_i)\}_{i=1}^n$ and $k(x)$ a vector with correlations to the new point $x$, to evaluate the functional given by the GP and get the mean at the points $x$  we do:
\begin{align}
\Sigma &:= \{K(x_i, x_j)\}_{ij} \\
k(x) &:= \{K(x,x_i)\}_{i=1}^n \\
f(x) &:= k(x)^t \Sigma^{-1}y \\
&= \sum_{i=1}^n K(x, x_i) \Sigma_{i}^{-1}y,
\end{align}
where $\Sigma_i^{-1}$ is the $i$'th row of the inverse correlation matrix (of course, we are just solving $\Sigma z = y$ and take the $i$'th component of $z$, so I will just use $z_i$ as a shorthand:
$$ f(x) = \sum_{i=1}^n z_i K(x, x_i)$$

I'll use wolframalpha to derive integrands. We also define a shorthand
$$
c_d := \sqrt{\theta_{1d}^2 + \epsilon}
$$

For the [constant](https://www.wolframalpha.com/input/?i=integral+exp%28-%28c%5E2%29%28x-y%29%5E2%29dx+from+0+to+1) we have:
\begin{align}
f_0 &= \int_{[0,1]^D} f(x) dx = \int_{[0,1]^D} \sum_{i=1}^n z_i K(x, x_i) dx \\
&= \sum_{i=1}^n z_i \theta_0^2 \prod_{d=1}^D \int_{0}^{1} \exp \Bigl(-c_d^2(x_d-x_{id})^2\Bigr) dx_d \\
&= \theta_0^2 \sum_{i=1}^n z_i \prod_{d=1}^D \frac{\sqrt{\pi}\Bigl[\text{erf}\Bigl(c_d-c_d x_{id})\Bigr) + \text{erf}\Bigl(c_d x_{id}\Bigr)\Bigr]}{2c_d}
\end{align}

For the [total variance](https://www.wolframalpha.com/input/?i=integral+exp%28-%28c%5E2%29%28%28x-y%29%5E2%2B%28x-z%29%5E2%29%29dx+from+a+to+b+) we have:
\begin{align}
\sigma^2(f) &= \int_{[0,1]^D}\Bigl(f(x) - f_0\Bigr)^2 dx = 
\int_{[0,1]^D} \Bigl(\sum_{i=1}^n z_i K(x, x_i) - f_0 \Bigr)^2 dx \\
&= \int_{[0,1]^D} \sum_{i=1}^n \sum_{j=1}^n z_i z_j K(x, x_i) K(x, x_j) dx - 2f_0 \int_{[0,1]^D}\sum_{i=1}^n z_i K(x, x_i) dx + f_0^2\int_{[0,1]^D}dx \\
&= f_0^2\Bigl(1 - 2\Bigr) + \theta_0^4 \sum_{i=1}^n \sum_{j=1}^n z_i z_j \prod_{d=1}^D \int_{0}^{1} \exp\Bigl(-c_d^2 \Bigl[(x_d - x_{id})^2 + (x_d - x_{jd})^2\Bigr]\Bigr)dx_d \\
&= f_0^2(\ldots) + \theta_0^4 \sum_{i=1}^n \sum_{j=1}^n z_i z_j \prod_{d=1}^D \frac{\sqrt{\frac{\pi}{2}} \exp\Bigl(-1/2c_d^2(x_{id}-x_{jd})^2\Bigr)\Bigl[\text{erf}\Bigl(\frac{c_d(x_{id}+x_{jd})}{\sqrt{2}}\Bigr)-\text{erf}\Bigl(\frac{c_d(-2 + x_{id} + x_{jd})}{\sqrt{2}}\Bigr)\Bigr]}{2c_d}
\end{align}

For the main effects we have:
\begin{align}
f_t(x_t) &= \theta_0^2 \sum_{i=1}^n z_i \exp\Bigl(-c_t^2(x_t-x_{it})^2\Bigr)\prod_{d\neq t}^D \frac{\sqrt{\pi}\Bigl[\text{erf}\Bigl(c_d(1-x_{id})\Bigr) - \text{erf}\Bigl(c_d(-x_{id})\Bigr)\Bigr]}{2c_d} - f_0\\ 
&=: \theta_0^2 \sum_{i=1}^n z_i \exp\Bigl(-c_t^2(x_t-x_{it})^2\Bigr) P_i - F_0 \\
\sigma^2(f_t(x_t)) &= \int_{[0,1]^D} \Bigl(f_{d}(x_{d})\Bigr)^2 dx = \int_{[0,1]^D} \Bigl(\theta_0^2 \sum_{i=1}^n z_i \exp\Bigl(-c_t^2(x_t-x_{it})^2\Bigr) P_i - F_0\Bigr)^2 dx \\
&= \int_{[0,1]^D} \theta_0^4 \sum_{i=1}^n \sum_{j=1}^n z_i z_j P_i P_j \exp\Bigl(-c_t \Bigl[(x_t - x_{it})^2 + (x_t - x_{jt})^2\Bigl]\Bigl) \\ 
&- 2 \theta_0^2 F_0 \sum_{i=1}^n z_i P_i \int_{[0,1]^D} \exp\Bigl(-c_t^2(x_t-x_{it})^2\Bigr) dx + F_0^2 \int_{[0,1]^D} dx \\
&= \theta_0^4 \sum_{i=1}^n \sum_{j=1}^n z_i z_j P_i P_j \frac{\sqrt{\frac{\pi}{2}} e^{-1/2c_t^2(x_{it}-x_{jt})^2}\Bigl[\text{erf}\Bigl(\frac{c_t(x_{it}+x_{jt})}{\sqrt{2}}\Bigr)-\text{erf}\Bigl(\frac{c_t(-2 + x_{it} + x_{jt})}{\sqrt{2}}\Bigr)\Bigl]}{2c_t} \\
&- 2\theta_0^2 F_0 \sum_{i=1}^n z_i P_i \frac{\sqrt{\pi}\Bigl[\text{erf}\Bigl(c_t(1-x_{it})\Bigr) - \text{erf}\Bigl(c_t(0-x_{it})\Bigr)\Bigr]}{2c_t}  + F_0^2 
\end{align}

# Numeric verification

In [1]:
import sklearn
from sklearn.gaussian_process.kernels import RBF
from sklearn.datasets import make_regression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import pairwise_kernels
import numpy as np
from scipy.special import erf
from scipy.integrate import dblquad

In [2]:
## Define GP and use normalization
dim = 2
theta_0 = 0.3
theta_1 = np.array([2.0]*dim)
ε = 1e-4
c = np.sqrt(theta_1**2 + ε)
kernel_test = lambda x, y: theta_0 * np.exp(-(x-y)**2@c**2)
length_scale = 1.0/(np.sqrt(2)*c)
kernel = RBF(length_scale = length_scale)

In [3]:
X, y = make_regression(n_samples=20, n_features=dim, n_informative=dim, bias=0.5, random_state=1000)

X_norm = MinMaxScaler().fit_transform(X)

X_train = X_norm[:10, :]
y_train = y[:10]
X_test = X_norm[10:,:]
y_test = y[10:]

In [4]:
K = theta_0**2*pairwise_kernels(X_train, metric=kernel)
z = np.linalg.solve(K, y_train)

def kernel_eval(X):
    return theta_0**2*pairwise_kernels(X, X_train, metric=kernel) @ z

K_test = pairwise_kernels(X_train, metric=kernel_test)
z_test = np.linalg.solve(K_test, y_train)
def kernel_test_eval(X):
    return pairwise_kernels(X, X_train, metric=kernel_test) @ z

In [5]:
y_eval = theta_0**2 * pairwise_kernels(X_test, X_train, metric=kernel) @ z
print(y_eval)
print(kernel_eval(X_test))
print(kernel_test_eval(X_test))

[-61.30361206 -57.98808062  22.53900295 -24.28231192 -16.72602774
  -2.12929409 -41.86531635  44.5630799   41.67531984 -12.97457399]
[-61.30361206 -57.98808062  22.53900295 -24.28231192 -16.72602774
  -2.12929409 -41.86531635  44.5630799   41.67531984 -12.97457399]
[-204.34537353 -193.29360207   75.13000982  -80.94103974  -55.7534258
   -7.09764696 -139.55105451  148.54359967  138.9177328   -43.24857997]


In [6]:
np.prod(np.sqrt(np.pi)/(2*c)* erf(c*(1-X_train)) + erf(c*X_train), axis=1)

array([0.91226262, 0.97676297, 1.35994095, 1.25343965, 1.52405704,
       1.51234951, 1.08494107, 1.18481574, 0.67870414, 1.37359053])

In [7]:
np.sqrt(np.pi)/(2*c)* (erf(c*(1-X_train)) + erf(c*X_train))

array([[0.74524907, 0.57208312],
       [0.64781443, 0.60612862],
       [0.74680421, 0.72391626],
       [0.74265738, 0.70187627],
       [0.7429866 , 0.74143716],
       [0.74660759, 0.73705457],
       [0.67494038, 0.54701156],
       [0.67883566, 0.74305994],
       [0.62949642, 0.50133195],
       [0.70611856, 0.5676102 ]])

## Contant $f_0$

In [8]:
# Constant analytic
f_0 = theta_0**2 * z.T @ np.prod(np.sqrt(np.pi)/(2*c)* (erf(c*(1-X_train)) + erf(c*X_train)), axis=1)
f_0

-2.1358132079939054

In [9]:
# Constant MC
for n in 10**np.array([1,2,3,4,5, 6, 7]):
    X = np.random.uniform(size=2*n).reshape((-1,dim))
    sol = np.mean(kernel_eval(X))
    print('%10d value = %.6f rel_error = %.6f' % (n, sol, np.abs(sol-f_0)/f_0))

        10 value = -12.693817 rel_error = -4.943318
       100 value = 4.239582 rel_error = -2.984997
      1000 value = -5.003225 rel_error = -1.342539
     10000 value = -1.203545 rel_error = -0.436493
    100000 value = -2.029017 rel_error = -0.050002
   1000000 value = -2.113715 rel_error = -0.010347
  10000000 value = -2.142167 rel_error = -0.002975


## Total variance $\sigma^2(f)$

In [ ]:
np.sqrt(np.pi/2)/(2*c)*pairwise_kernels(X_train, metric=RBF(length_scale=1/c)) 

In [ ]:
sigma2 = f_0**2 + theta_0^2*z.T @ z.T @ np.prod()